In [ ]:
@file:DependsOn("com.toldoven.aoc:aoc-kotlin-notebook:1.1.2")

In [ ]:
import com.toldoven.aoc.notebook.AocClient

val client = AocClient.fromFile()

In [ ]:
val day = client.interactiveDay(2025, 8)
day.viewPartOne()

In [ ]:
val input = day.input()
input

In [ ]:
val exampleInput = """
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
""".trimIndent()

In [ ]:
data class Point3D(val x: Int, val y: Int, val z: Int)

fun List<Int>.toPoint3D(): Point3D {
    require(count() == 3) { "A 3D Point requires exactly 3 coordinates - got ${count()}" }
    return Point3D(this[0], this[1], this[2])
}

fun String.toPoint3D() = split(",").map(String::toInt).toPoint3D()

fun String.parseInput() = lines().map { it.toPoint3D() }

In [ ]:
val examplePoints = exampleInput.parseInput()

In [ ]:
val points = day.input().parseInput()
points

In [ ]:
fun Int.pow(exponent: Int): Long {
    var result: Long = 1
    repeat(exponent) {
        result *= this
    }
    return result
}

fun Point3D.distanceTo(other: Point3D) = (x - other.x).pow(2) + (y - other.y).pow(2) + (z - other.z).pow(2)

In [ ]:
examplePoints.map {
    it to examplePoints.map { other -> it.distanceTo(other) }
}.joinToString("\n")

In [ ]:
points.map {
    it to points.map { other -> it.distanceTo(other) }
}.joinToString("\n")

In [ ]:
examplePoints
    .flatMapIndexed { index, basePoint ->
        examplePoints
            .drop(index)
            .map { other -> basePoint.distanceTo(other) to (basePoint to other) }
            .sortedBy { it.first }
            .drop(1)
    }
    .sortedBy { it.first }
    .joinToString("\n")

In [ ]:
data class Circuit(
    val junctions: Set<Point3D>,
)

In [ ]:
fun Circuit.minDistanceTo(other: Circuit): Long {
    return junctions
        .minOf {
            other
                .junctions
                .minOf { otherJunction -> it.distanceTo(otherJunction) }
        }
}

In [ ]:
val exampleCircuits = examplePoints.map { Circuit(setOf(it)) }

exampleCircuits
    .flatMapIndexed { index, baseCircuit ->
        exampleCircuits
            .drop(index)
            .map { other -> baseCircuit.minDistanceTo(other) to (baseCircuit to other) }
            .sortedBy { it.first }
            .drop(1)
    }
    .sortedBy { it.first }
    .joinToString("\n")

In [ ]:
fun Circuit.merge(other: Circuit): Circuit = Circuit(junctions + other.junctions)

In [ ]:
fun Iterable<Circuit>.getClosestCircuits(): Pair<Circuit, Circuit> {
    return flatMapIndexed { index, baseCircuit ->
            map { other -> baseCircuit.minDistanceTo(other) to (baseCircuit to other) }
            .sortedBy { it.first }
            .drop(1)
    }
        .sortedBy { it.first }
        .first()
        .second
}

In [ ]:
exampleCircuits.getClosestCircuits()

In [ ]:
(0..<10)
    .fold(exampleCircuits.toSet()) { circuits, idx ->
        println("=== $idx ===")
        val closestCircuits = circuits.getClosestCircuits()
        val merged = circuits.toSet() - closestCircuits.first - closestCircuits.second + (closestCircuits.first.merge(closestCircuits.second))

        println("Merged $closestCircuits - stats:")
        merged.groupBy { it.junctions.size }.forEach { (size, circuits) -> println("$size: ${circuits.size}") }

        merged
    }
    .count()



In [ ]:
exampleCircuits.joinToString("\n")

In [ ]:
Circuit(setOf(Point3D(0, 0, 0))).let {
    it.minDistanceTo(it)
}

In [ ]:
data class Junction(val coordinates: Point3D) {
    var connections: Set<Junction> = emptySet()

    fun connectTo(other: Junction) {
        connections = connections + other
        other.connections = other.connections + this
    }

    fun isConnectedTo(other: Junction) = connections.find { it.coordinates == other.coordinates } != null

    fun distanceTo(other: Junction) = coordinates.distanceTo(other.coordinates)
}

In [ ]:
val a = Junction(Point3D(0, 0, 0))
val b = Junction(Point3D(1, 0, 0))
a.connectTo(b)
a.isConnectedTo(b)
b.isConnectedTo(a)

In [ ]:
data class Circuit2(val junctions: Set<Junction>) {
    fun minDistanceTo(other: Circuit2): Long {
        return junctions
            .minOf {
                other
                    .junctions
                    .minOf { otherJunction -> it.distanceTo(otherJunction) }
            }
    }

    fun merge(other: Circuit2): Circuit2 = Circuit2(junctions + other.junctions)
}

In [ ]:
val exampleCircuits2 = exampleInput.parseInput().map { Circuit2(setOf(Junction(it))) }
exampleCircuits2.joinToString("\n")

In [ ]:
fun Iterable<Junction>.getClosestJunctions(): Pair<Junction, Junction> {
    return flatMapIndexed { index, startingJunction ->
        drop(index)
            .filterNot { it == startingJunction || it.isConnectedTo(startingJunction)}
            .map { other -> startingJunction.distanceTo(other) to (startingJunction to other) }
            .sortedBy { it.first }
    }
        .sortedBy { it.first }
        .first()
        .second
}

In [ ]:
fun Iterable<Circuit2>.connectClosestJunctions(iterations: Int): Set<Circuit2> {
    forEach { it.junctions.forEach { it.connections = emptySet()} }

    return (0..<iterations)
        .fold(toSet()) { circuits, idx ->
            val junctionToCurcuit = circuits.flatMap { circuit -> circuit.junctions.map { it to circuit } }.toMap()


            //println("=== $idx ===")
            val (first, second) = junctionToCurcuit.keys.getClosestJunctions()
            first.connectTo(second)

            val firstCircuit = junctionToCurcuit.getValue(first)
            val secondCircuit = junctionToCurcuit.getValue(second)
            val merged = circuits - firstCircuit - secondCircuit + (firstCircuit.merge(secondCircuit))

            //println("Connected $first with $second - stats:")
            //println("Connected? ${first.isConnectedTo(second)} || ${second.isConnectedTo(first)}")
            //merged.groupBy { it.junctions.size }.forEach { (size, circuits) -> println("$size: ${circuits.size}") }

            merged
        }
}

In [ ]:
fun List<Point3D>.getSortedByMinDistance(): List<Pair<Point3D, Point3D>> {
    return flatMapIndexed { index, startingJunction ->
        drop(index)
            .filterNot { it == startingJunction }
            .map { other -> startingJunction.distanceTo(other) to (startingJunction to other) }
    }
        .sortedBy { it.first }
        .map { it.second }
}

In [ ]:
exampleCircuits2.connectClosestJunctions(10).count()

In [ ]:
fun List<Point3D>.connectJunctions(iterations: Int): Set<Circuit2> {
    val sortedPoints = getSortedByMinDistance()
    val junctions = map { Junction(it) }
    val coordinateToJunctions = junctions.associateBy { it.coordinates }

    return sortedPoints.take(iterations).fold(
        map { Circuit2(setOf(Junction(it))) }.toSet()
    ) { circuits, (first, second) ->
        val coordinateToCircuit: Map<Point3D, Circuit2> = circuits.flatMap { circuit -> circuit.junctions.map { it.coordinates to circuit } }.toMap()

        coordinateToJunctions[first]!!.connectTo(coordinateToJunctions[second]!!)

        circuits - coordinateToCircuit[first]!! - coordinateToCircuit[second]!! + (coordinateToCircuit[first]!!.merge(coordinateToCircuit[second]!!))
    }
}

In [ ]:
examplePoints.connectJunctions(10).count()

In [ ]:
examplePoints.connectJunctions(10).sortedByDescending { it.junctions.size }.take(3).fold(1) { acc, circuit -> acc * circuit.junctions.size }

In [ ]:
val part1Solution = points.connectJunctions(1000).sortedByDescending { it.junctions.size }.take(3).fold(1) { acc, circuit -> acc * circuit.junctions.size }
part1Solution

In [ ]:
day.submitPartOne(part1Solution)

In [ ]:
day.viewPartTwo()

In [ ]:
fun List<Point3D>.collapse(): Long  {
    val sortedPoints = getSortedByMinDistance()
    val junctions = map { Junction(it) }
    val coordinateToJunctions = junctions.associateBy { it.coordinates }

    sortedPoints.fold(
        map { Circuit2(setOf(Junction(it))) }.toSet()
    ) { circuits, (first, second) ->
        val coordinateToCircuit: Map<Point3D, Circuit2> = circuits.flatMap { circuit -> circuit.junctions.map { it.coordinates to circuit } }.toMap()

        coordinateToJunctions[first]!!.connectTo(coordinateToJunctions[second]!!)

        val merged = circuits - coordinateToCircuit[first]!! - coordinateToCircuit[second]!! + (coordinateToCircuit[first]!!.merge(coordinateToCircuit[second]!!))

        if (merged.count() == 1) {
            return 1L * first.x * second.x
        }

        merged
    }

    throw IllegalStateException("Failed to determine result")
}

In [ ]:
examplePoints.collapse()

In [ ]:
val part2Solution = points.collapse()
part2Solution

In [ ]:
day.submitPartTwo(part2Solution)